In [1]:
import oda_api.token 
import logging
import numpy as np
from oda_api.api import DispatcherAPI
from oda_api.plot_tools import OdaImage, OdaLightCurve, OdaSpectrum
import matplotlib.pyplot as plt
import astroquery.heasarc
from astropy.wcs import WCS
from astropy.io import fits
from astroquery.simbad import Simbad
from astropy import coordinates as coord
from astropy.coordinates import SkyCoord
from astropy.time import Time
from matplotlib.patches import Circle
from astroquery.jplhorizons import Horizons
import pandas as pd
import astropy.units as u
import json
from collections import defaultdict
from astropy.io import ascii

In [2]:
logging.getLogger().setLevel(logging.WARNING)
logging.getLogger('oda_api').addHandler(logging.StreamHandler())

Load the ScWs

In [ ]:
'''scw_ids = []
scw_versions = []
scw_start_times = []
scw_end_times = []
jupiter_ra = []
jupiter_dec = []

with open("../data/Jupiter-ScWs.txt", "r") as f:
    next(f)
    for line in f:
        parts = line.strip().split(", ")
        scw_ids.append(parts[0])
        scw_versions.append(parts[1])
        scw_start_times.append(float(parts[2]))  
        scw_end_times.append(float(parts[3]))  
        jupiter_ra.append(float(parts[4]))  
        jupiter_dec.append(float(parts[5]))  

unique_sorted_data = {}
for sid, ver, start, end, ra, dec in sorted(zip(scw_ids, scw_versions, scw_start_times, scw_end_times, jupiter_ra, jupiter_dec), key=lambda x: x[0]):
    if sid not in unique_sorted_data:  
        unique_sorted_data[sid] = (sid, ver, start, end, ra, dec)

scw_ids, scw_versions, start, end, jupiter_ra, jupiter_dec = map(list, zip(*unique_sorted_data.values()))

scw = [id + "." + ver for id, ver in zip(scw_ids, scw_versions)]
durations = [(e - s) for e,s in zip(end, start)]'''

In [3]:
jupiter_table = ascii.read('../data/jupiter_table.dat')

scw_ids = jupiter_table['scw_id'].data
scw_versions = jupiter_table['scw_ver'].data
scw = [str(id).zfill(12) + ".00" + str(ver) for id, ver in zip(scw_ids, scw_versions)]

start = jupiter_table['start_date'].data
end = jupiter_table['end_date'].data
durations = [(e - s) for e,s in zip(end, start)]

jupiter_ra = jupiter_table['jupiter_ra'].data
jupiter_dec = jupiter_table['jupiter_dec'].data

## QUERYING 

## IMAGES

In [4]:
isot_start_times = Time(start, format='mjd').isot
isot_end_times = Time(end, format='mjd').isot
duration_seconds = [duration * 86400 for duration in durations]
year_months = [st[:7] for st in isot_start_times]

How many ScWs after filtering for pointings?

In [5]:
scw_count_by_year_month = {}

for year_month in sorted(set(year_months)):
    filtered_scws = [
        (scw[i], isot_start_times[i], isot_end_times[i], jupiter_ra[i], jupiter_dec[i])
        for i, year_month_in_list in enumerate(year_months)
        if year_month_in_list == year_month
    ]

    if not filtered_scws:
        scw_count_by_year_month[year_month] = 0
        continue

    rrrr_max_pppp = defaultdict(int)
    for scwgr in filtered_scws:
        rrrr = scwgr[0][:4]  
        pppp = int(scwgr[0][4:8])
        rrrr_max_pppp[rrrr] = max(rrrr_max_pppp[rrrr], pppp)

    filtered_scws = [
        scwind for scwind in filtered_scws
        if 5 < int(scwind[0][4:8]) < rrrr_max_pppp[scwind[0][:4]] - 10
    ]

    scw_count_by_year_month[year_month] = len(filtered_scws)

print(f"Total SCWs after filtering: {sum(scw_count_by_year_month.values())}")


Total SCWs after filtering: 109


15 - 30 keV

In [6]:
from collections import defaultdict

disp_by_date = {}
data_by_date = {}
successful_scws = []

while True:
    image_results = []

    # Prepare SCWs with relevant data
    filtered_scws = [
        (scw[i], isot_start_times[i], isot_end_times[i], jupiter_ra[i], jupiter_dec[i])
        for i in range(len(scw))
    ]

    # Determine the maximum SCW index per revolution
    rrrr_max_pppp = defaultdict(int)
    for scw_id, *_ in filtered_scws:
        rrrr = scw_id[:4]
        pppp = int(scw_id[4:8])
        rrrr_max_pppp[rrrr] = max(rrrr_max_pppp[rrrr], pppp)

    # Apply filtering (exclude first 5 and last 10 SCWs per revolution)
    filtered_scws = [
        (scw_id, start_time, end_time, jra, jdec)
        for scw_id, start_time, end_time, jra, jdec in filtered_scws
        if 5 < int(scw_id[4:8]) < rrrr_max_pppp[scw_id[:4]] - 10
    ]

    if not filtered_scws:
        print("No SCWs found after filtering.")
    else:
        print(f"{len(filtered_scws)} SCWs found after filtering.")

    # Process each SCW directly in a single loop
    for scw_id, start_time, end_time, jra, jdec in filtered_scws:
        print(f"Trying SCW {scw_id}")

        par_dict = {
            "RA": jra,
            "DEC": jdec,
            "E1_keV": "15",
            "E2_keV": "30",
            "T_format": "isot",
            "T1": start_time,
            "T2": end_time,
            "detection_threshold": "5",
            "instrument": "isgri",
            "osa_version": "OSA11.2",
            "product": "isgri_image",
            "product_type": "Real",
            "scw_list": [scw_id],
        }

        if scw_id not in disp_by_date:
            disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", wait=False)

        _disp = disp_by_date[scw_id]
        data = data_by_date.get(scw_id, None)

        if data is None and not _disp.is_failed:
            try:
                if not _disp.is_submitted:
                    data = _disp.get_product(**par_dict, silent=True)
                else:
                    _disp.poll()

                print("Is complete ", _disp.is_complete)
                if _disp.is_complete:
                    data = _disp.get_product(**par_dict, silent=True)
                    data_by_date[scw_id] = data

            except Exception as e:
                print(f"Query failed for SCW {scw_id}: {e}")
                continue

        successful_scws.append(scw_id)
        image_results.append(data)


    # Summary of completed SCWs
    n_complete = sum(1 for _disp in disp_by_date.values() if _disp.is_complete)
    print(f"complete {n_complete} / {len(disp_by_date)}")

    if n_complete == len(disp_by_date):
        print("done!")
        break
    else:
        print("not done")
        

109 SCWs found after filtering.
Trying SCW 066800190010.001
Is complete  True
Trying SCW 066800200010.001
Is complete  True
Trying SCW 066800220010.001
Is complete  True
Trying SCW 066800230010.001
Is complete  True
Trying SCW 066800240010.001
Is complete  True
Trying SCW 066800250010.001
Is complete  True
Trying SCW 066800270010.001
Is complete  True
Trying SCW 066800290010.001
Is complete  True
Trying SCW 066800300010.001
Is complete  True
Trying SCW 066800310010.001
Is complete  True
Trying SCW 066800320010.001
Is complete  True
Trying SCW 066800330010.001
Is complete  True
Trying SCW 066800340010.001
Is complete  True
Trying SCW 066800370010.001
Is complete  True
Trying SCW 066800380010.001
Is complete  True
Trying SCW 066800390010.001
Is complete  True
Trying SCW 066800410010.001
Is complete  True
Trying SCW 066800440010.001
Is complete  True
Trying SCW 066800450010.001
Is complete  True
Trying SCW 066800460010.001
Is complete  True
Trying SCW 066800470010.001
Is complete  True
Tr

In [ ]:
# We already have the successful SCWs from the previous codes, so we can use them
import os

scw_to_match1 = []
directory1 = "../data/JupiterIMG_FITS_15_30/"
for filename in os.listdir(directory1):
    if filename.endswith(".fits"):
        scw_to_match1.append(filename[:16])

scw_to_match2 = []
directory2 = "../data/JupiterIMG_FITS_30_60/"
for filename in os.listdir(directory2):
    if filename.endswith(".fits"):
        scw_to_match2.append(filename[:16])

scw_to_match3 = []
directory3 = "../data/JupiterIMG_FITS_3_15/"
for filename in os.listdir(directory3):
    if filename.endswith(".fits"):
        scw_to_match3.append(filename[:16])

print("Number of SCWs in 15-30 keV: ", len(scw_to_match1))
print("Number of SCWs in 30-60 keV: ", len(scw_to_match2))
print("Number of SCWs in 3-15 keV: ", len(scw_to_match3))

print("Number of SCWs left to process for 3-15 keV: ", len(scw_to_match1) - len(scw_to_match3))
print("Number of SCWs left to process for 30-60 keV: ", len(scw_to_match1) - len(scw_to_match2))

scw_to_process = [] # for 3-15 keV
for scw_id in scw_to_match2:
    if scw_id not in scw_to_match1: # change 3 to 2 for 30-60 keV
        scw_to_process.append(scw_id)
print("Number of SCWs to process: ", len(scw_to_process))

30 - 60 keV

In [ ]:
from collections import defaultdict

disp_by_date = {}
data_by_date = {}
successful_scws = []

while True:
    image_results = []

    # Prepare SCWs with relevant data
    filtered_scws = [
        (scw[i], isot_start_times[i], isot_end_times[i], jupiter_ra[i], jupiter_dec[i])
        for i in range(len(scw))
    ]

    # Determine the maximum SCW index per revolution
    rrrr_max_pppp = defaultdict(int)
    for scw_id, *_ in filtered_scws:
        rrrr = scw_id[:4]
        pppp = int(scw_id[4:8])
        rrrr_max_pppp[rrrr] = max(rrrr_max_pppp[rrrr], pppp)

    # Apply filtering (exclude first 5 and last 10 SCWs per revolution)
    filtered_scws = [
        (scw_id, start_time, end_time, jra, jdec)
        for scw_id, start_time, end_time, jra, jdec in filtered_scws
        if 5 < int(scw_id[4:8]) < rrrr_max_pppp[scw_id[:4]] - 10
    ]

    if not filtered_scws:
        print("No SCWs found after filtering.")
    else:
        print(f"{len(filtered_scws)} SCWs found after filtering.")

    # Process each SCW directly in a single loop
    for scw_id, start_time, end_time, jra, jdec in filtered_scws:

        if scw_id not in scw_to_process:
            continue

        print(f"Trying SCW {scw_id}")

        par_dict = {
            "RA": jra,
            "DEC": jdec,
            "E1_keV": "30",
            "E2_keV": "60",
            "T_format": "isot",
            "T1": start_time,
            "T2": end_time,
            "detection_threshold": "5",
            "instrument": "isgri",
            "osa_version": "OSA11.2",
            "product": "isgri_image",
            "product_type": "Real",
            "scw_list": [scw_id],
        }

        if scw_id not in disp_by_date:
            disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", wait=False)

        _disp = disp_by_date[scw_id]
        data = data_by_date.get(scw_id, None)

        if data is None and not _disp.is_failed:
            try:
                if not _disp.is_submitted:
                    data = _disp.get_product(**par_dict, silent=True)
                else:
                    _disp.poll()

                print("Is complete ", _disp.is_complete)
                if _disp.is_complete:
                    data = _disp.get_product(**par_dict, silent=True)
                    data_by_date[scw_id] = data

            except Exception as e:
                print(f"Query failed for SCW {scw_id}: {e}")
                continue

        successful_scws.append(scw_id)
        image_results.append(data)


    # Summary of completed SCWs
    n_complete = sum(1 for _disp in disp_by_date.values() if _disp.is_complete)
    print(f"complete {n_complete} / {len(disp_by_date)}")

    if n_complete == len(disp_by_date):
        print("done!")
    else:
        print("not done")


3 - 15 keV

In [ ]:
disp_by_date = {} 
data_by_date = {}
successful_scws = []

disp = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", instrument="mock")

while True:
    image_results = []

    for year_month in sorted(set(year_months)):

        filtered_scws = [
            (scw[i], isot_start_times[i], isot_end_times[i], duration_seconds[i], jupiter_ra[i], jupiter_dec[i])
            for i, year_month_in_list in enumerate(year_months)
            if year_month_in_list == year_month
        ]

        if not filtered_scws:
            # print(f"No SCWs found for {year_month}")
            continue

        # Add filtering for not choosing the first 5 and last 10 SCWs of each revolution
        rrrr_max_pppp = defaultdict(int)
        for scwgr in filtered_scws:
            rrrr = scwgr[0][:4]  
            pppp = int(scwgr[0][4:8])
            rrrr_max_pppp[rrrr] = max(rrrr_max_pppp[rrrr], pppp)

        filtered_scws = [
            scwind for scwind in filtered_scws
            if 5 < int(scwind[0][4:8]) < rrrr_max_pppp[scwind[0][:4]] - 10
        ]

        if not filtered_scws:
            # print(f"No SCWs found for {year_month} after filtering for pointings")
            continue
        # else:
            # print(f"{len(filtered_scws)} SCWs found for {year_month} after filtering for pointings")

        # Process SCWs for the current year-month
        for scw_id, start_time, end_time, duration, ra, dec in filtered_scws:

            print(f"Trying SCW {scw_id} for {year_month}")

            par_dict = {
                "RA": ra,
                "DEC": dec,
                "E1_keV": "3",
                "E2_keV": "15",
                "T_format": "isot",
                'T1': start_time,
                'T2': end_time,
                "detection_threshold": "5",
                "instrument": "jemx",
                "osa_version": "OSA11.2",
                "product": "jemx_image",
                "product_type": "Real",
                "scw_list": [scw_id],
            }

            if scw_id not in disp_by_date:
                disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", instrument="mock", wait=False)

            _disp = disp_by_date[scw_id]

            data = data_by_date.get(scw_id, None)

            if data is None and not _disp.is_failed:
                try:
                    if not _disp.is_submitted:
                        data = _disp.get_product(**par_dict, silent=True)
                    else:
                        _disp.poll()

                    if not _disp.is_complete:
                        break  # Continue to the next SCW if not complete

                    data = _disp.get_product(**par_dict, silent=True)
                    data_by_date[scw_id] = data
                    image_results.append(data)
                    successful_scws.append(scw_id)
                    # print(f"Query successful for SCW {scw_id}")
                    continue

                except Exception as e:
                    # print(f"Query failed for SCW {scw_id}: {e}")
                    continue  # Try the next SCW

    # Now, poll all SCWs that have been submitted
    print("Polling for completion of all SCWs.")
    n_complete = 0

    # Poll all SCWs that have been submitted (skip already completed ones)
    for scw_id, _disp in disp_by_date.items():
        if _disp.is_complete:
            try:
                if scw_id not in data_by_date:
                    data = _disp.get_product(silent=True)
                    data_by_date[scw_id] = data
                else:
                    data = data_by_date[scw_id]  
                
                image_results.append(data)  
                successful_scws.append(scw_id)
                print(f"Data retrieved for SCW {scw_id}")

            except Exception as e:
                print(f"Error retrieving data for SCW {scw_id}: {e}")

        if _disp.is_complete:
            n_complete += 1

    print(f"Complete {n_complete} / {len(disp_by_date)}")

    if n_complete == len(disp_by_date):
        print("All SCWs complete! Done.")
        break
    print("Not done, waiting for SCWs to finish...")

    

In [ ]:
from collections import defaultdict

disp_by_date = {}
data_by_date = {}
successful_scws = []

while True:
    image_results = []

    # Prepare SCWs with relevant data
    filtered_scws = [
        (scw[i], isot_start_times[i], isot_end_times[i], jupiter_ra[i], jupiter_dec[i])
        for i in range(len(scw))
    ]

    # Determine the maximum SCW index per revolution
    rrrr_max_pppp = defaultdict(int)
    for scw_id, *_ in filtered_scws:
        rrrr = scw_id[:4]
        pppp = int(scw_id[4:8])
        rrrr_max_pppp[rrrr] = max(rrrr_max_pppp[rrrr], pppp)

    # Apply filtering (exclude first 5 and last 10 SCWs per revolution)
    filtered_scws = [
        (scw_id, start_time, end_time, jra, jdec)
        for scw_id, start_time, end_time, jra, jdec in filtered_scws
        if 5 < int(scw_id[4:8]) < rrrr_max_pppp[scw_id[:4]] - 10
    ]

    if not filtered_scws:
        print("No SCWs found after filtering.")
    else:
        print(f"{len(filtered_scws)} SCWs found after filtering.")

    # Process each SCW directly in a single loop
    for scw_id, start_time, end_time, jra, jdec in filtered_scws:

        if scw_id not in scw_to_process:
            continue

        print(f"Trying SCW {scw_id}")

        par_dict = {
            "RA": jra,
            "DEC": jdec,
            "E1_keV": "3",
            "E2_keV": "15",
            "T_format": "isot",
            "T1": start_time,
            "T2": end_time,
            "detection_threshold": "5",
            "instrument": "jemx",
            "osa_version": "OSA11.2",
            "product": "jemx_image",
            "product_type": "Real",
            "scw_list": [scw_id],
        }

        if scw_id not in disp_by_date:
            disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", wait=False)

        _disp = disp_by_date[scw_id]
        data = data_by_date.get(scw_id, None)

        if data is None and not _disp.is_failed:
            try:
                if not _disp.is_submitted:
                    data = _disp.get_product(**par_dict, silent=True)
                else:
                    _disp.poll()

                print("Is complete ", _disp.is_complete)
                if _disp.is_complete:
                    data = _disp.get_product(**par_dict, silent=True)
                    data_by_date[scw_id] = data

            except Exception as e:
                print(f"Query failed for SCW {scw_id}: {e}")
                continue

        successful_scws.append(scw_id)
        image_results.append(data)


    # Summary of completed SCWs
    n_complete = sum(1 for _disp in disp_by_date.values() if _disp.is_complete)
    print(f"complete {n_complete} / {len(disp_by_date)}")

    if n_complete == len(disp_by_date):
        print("done!")
    else:
        print("not done")


Save FITS

In [8]:
new_results = []
new_scws = []
for scw_id, data in data_by_date.items():
    if data is not None:
        new_results.append(data)
        new_scws.append(scw_id)

In [9]:
# Save FITS files
import os

output_dir = "../data/Jupiter/30-60keV/Images/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Check if the directory is empty
#if os.path.exists(output_dir) and os.listdir(output_dir):
   # print(f"Directory '{output_dir}' is not empty. Aborting to prevent overwriting.")
#else:
for i, data in enumerate(new_results):
    #if new_scws[i] in scw_to_match:
    im = OdaImage(data)
    im.write_fits(os.path.join(output_dir, f"{new_scws[i]}"))


## LIGHT CURVE

In [ ]:
api_cat={
    "cat_frame": "fk5",
    "cat_coord_units": "deg",
    "cat_column_list": [
        [0],
        ["Jupiter"],
        [125.4826889038086],
        [0], # this will be updated during the querying
        [0],
        [-32768],
        [2],
        [0],
        [0.0002800000074785203]],
    "cat_column_names": [
        "meta_ID",
        "src_names",
        "significance",
        "ra",
        "dec",
        "NEW_SOURCE",
        "ISGRI_FLAG",
        "FLAG",
        "ERR_RAD"
    ],
    "cat_column_descr":
        [
            ["meta_ID", "<i8"],
            ["src_names", "<U11"],
            ["significance", "<f8"],
            ["ra", "<f8"],
            ["dec", "<f8"],
            ["NEW_SOURCE", "<i8"],
            ["ISGRI_FLAG", "<i8"],
            ["FLAG", "<i8"],
            ["ERR_RAD", "<f8"]
        ],
    "cat_lat_name": "dec",
    "cat_lon_name": "ra"
}

In [ ]:
from collections import defaultdict

lc_disp_by_date = {}
lc_data_by_date = {}
successful_lc_scws = []
lc_results = []

filtered_scws = [
    (scw[i], isot_start_times[i], isot_end_times[i], duration_seconds[i], jupiter_ra[i], jupiter_dec[i])
    for i in range(len(scw))
]

rrrr_max_pppp = defaultdict(int)
for scw_id, *_ in filtered_scws:
    rrrr = scw_id[:4]
    pppp = int(scw_id[4:8])
    rrrr_max_pppp[rrrr] = max(rrrr_max_pppp[rrrr], pppp)

filtered_scws = [
    (scw_id, start_time, end_time, dur, jra, jdec)
    for scw_id, start_time, end_time, dur, jra, jdec in filtered_scws
    if 5 < int(scw_id[4:8]) < rrrr_max_pppp[scw_id[:4]] - 10
]

if not filtered_scws:
    print("No SCWs found after filtering.")
else:
    print(f"{len(filtered_scws)} SCWs found after filtering.")

# Process each SCW directly in a single loop
for scw_id, start_time, end_time, duration, ra, dec in filtered_scws:
    print(f"Trying SCW {scw_id}")

    api_cat.update({
        "cat_column_list": [
            [0],
            ["Jupiter"],
            [125.4826889038086],
            [ra],  # Update RA
            [dec],  # Update DEC
            [-32768],
            [2],
            [0],
            [0.0002800000074785203]
        ]
    })

    par_dict = {
        "RA": ra,
        "DEC": dec,
        "E1_keV": "15",
        "E2_keV": "30", 
        "T_format": "isot",
        'T1': start_time,
        'T2': end_time,
        "time_bin": duration, 
        "instrument": "isgri",
        "osa_version": "OSA11.2",
        "product": "isgri_lc",
        "product_type": "Real",
        "scw_list": [scw_id],
        #'token': disp.disable_email_token(oda_api.token.discover_token()),
        'selected_catalog': json.dumps(api_cat)
        }
    
    if scw_id not in lc_disp_by_date:
        lc_disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", wait=False)

    _disp = lc_disp_by_date[scw_id]
    data = lc_data_by_date.get(scw_id, None)

    if data is None and not _disp.is_failed:
        try:
            if not _disp.is_submitted:
                data = _disp.get_product(**par_dict, silent=True)
            else:
                _disp.poll()

            print("Is complete ", _disp.is_complete)
            if _disp.is_complete:
                data = _disp.get_product(**par_dict, silent=True)
                lc_data_by_date[scw_id] = data

        except Exception as e:
            print(f"Query failed for SCW {scw_id}: {e}")
            continue

    successful_lc_scws.append(scw_id)
    lc_results.append(data)

# Summary of completed SCWs
n_complete = sum(1 for _disp in lc_disp_by_date.values() if _disp.is_complete)
print(f"complete {n_complete} / {len(lc_disp_by_date)}")

if n_complete == len(lc_disp_by_date):
    print("done!")
else:
    print("not done")

Save FITS

In [ ]:
# Save FITS files
for i, data in enumerate(lc_results):
    lc = OdaLightCurve(data)
    lc.write_fits(f"../data/JupiterLC_FITS_15_30/{successful_lc_scws[i]}")